# **Dataset Analysis - Analyzing CIA Factbook Data Using SQL**

In this project, we'll work with data from the CIA World Factbook a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

- population : The population as of 2015.
- population_growth : The annual population growth rate, as a percentage.
- area : The total land and water area.

In this guided project, we'll use SQL to explore and analyze data from this database.
To download the SQLite database - [factbook.db](<https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db>)

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Data Overview 

In [3]:
%%sql
SELECT * FROM facts LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Here are the descriptions of the columns:

|Column Name|Description|
|:----|:----|
|name|The name of the country|
|area|The total land and sea area of the country.|
|population|The country's population.|
|population_growth|The country's population growth as a percentage.|
|birth_rate|The country's birth rate, or the number of births a year per 1,000 people.|
|death_rate|The country's death rate, or the number of death a year per 1,000 people.|
|area|The country's total area (both land and water).|
|area_land|The country's land area in square kilometers.|
|area_water|The country's waterarea in square kilometers.|

## Summary Statistics

In [4]:
%%sql
SELECT MAX(population) max_pop,MIN(population) min_pop,MAX(population_growth) max_pop_growth,MIN(population_growth) min_pop_growth
FROM facts;

Done.


max_pop,min_pop,max_pop_growth,min_pop_growth
7256490011,0,4.02,0.0


A few things stick out from the summary statistics in the last screen:
- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

## Exploring Outliers

In [5]:
%%sql
SELECT name, population
FROM facts 
WHERE population == (SELECT MAX(population) FROM facts)

Done.


name,population
World,7256490011


In [6]:
%%sql
SELECT name, population
FROM facts 
WHERE population == (SELECT MIN(population) FROM facts)

Done.


name,population
Antarctica,0


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0.

In [7]:
%%sql
SELECT name, population_growth
FROM facts 
WHERE population_growth == (SELECT MIN(population_growth) FROM facts)

Done.


name,population_growth
Holy See (Vatican City),0.0
Cocos (Keeling) Islands,0.0
Greenland,0.0
Pitcairn Islands,0.0


In [8]:
%%sql
SELECT name, population_growth
FROM facts 
WHERE population_growth == (SELECT MAX(population_growth) FROM facts)

Done.


name,population_growth
South Sudan,4.02


Thus from the results of above SQL queries we conclude that:

- the table contains a row for Antarctica, which explains having a value 0 in the population column.
- the table contains a row for the whole world, which explains having a value of over 7.2 billion in the population column.

We also found that Antarctica population growth is None. Obviously, this result is not surprising as there is no population. 

The table contains 4 countries which do have some population but numbers shows zero population growth in these countries. Our query above also reveals that South Sudan is the country which has maximum population growth rate of 4.02.

## Exploring average population and area

In [9]:
%%sql
SELECT AVG(population) pop_avg,AVG(area) area_avg
FROM facts
WHERE name <> 'World'

Done.


pop_avg,area_avg
32242666.56846473,555093.546184739


The result of above query reveals that the average population is far more important than the average area. Does this means overpopulated ?

To get a better idea, let's find out which countries are overpopulated, the more densely populated.

## Finding densely populated countries

In [10]:
%%sql
SELECT name, population, area
FROM facts
WHERE population > (SELECT AVG(population) FROM facts WHERE name <> 'World')
AND area < (SELECT AVG(area) FROM facts WHERE name <> 'World')

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


Above query outputs the name of the countries in the table having dense population. We identified them by selecting the countries that answered the two following criterias : 

- Above average values for population
- Below average values for area.

The result of our above queries shows that as per 2015 records there are seven countries which are overpopulated/densely populated.

## Which countries have the highest ratios of water to land ?

In [11]:
%%sql
SELECT name, CAST(area_water as Float)/CAST(area_land as Float) water_to_land_ratio 
FROM facts 
ORDER BY water_to_land_ratio DESC
LIMIT 5;

Done.


name,water_to_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623


The above query reveals that the country British Indian Ocean Territory has highest ratio of 905.67 of water to land followed by Virgin Islands with 4.52.

## Which countries have more water than land ? 

In [12]:
%%sql
SELECT name, area_water, area_land
FROM facts 
WHERE area_water > area_land

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


The above query explains in more details the ratios 905.67 and 4.52 of water to land for the countries British Indian Ocean Territory and Virgin Islands respectively.

## Which countries will add the most people to their population next year ?

In [13]:
%%sql
SELECT name, population_growth
FROM facts 
ORDER BY population_growth DESC
LIMIT 5;

Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24


Through the above query we collected the top 5 countries per population growth rate. South Sudan is leading with highest population growth rate of 4.02. These numbers are only valuable for 2016 as this is 2015 data records. 

## Which countries have a higher death rate than birth rate ?

In [17]:
%%sql
SELECT name, birth_rate, death_rate, ROUND((death_rate - birth_rate),3) rate_spread
FROM facts
WHERE death_rate > birth_rate
ORDER BY rate_spread DESC

Done.


name,birth_rate,death_rate,rate_spread
Bulgaria,8.92,14.44,5.52
Serbia,9.08,13.66,4.58
Latvia,10.0,14.31,4.31
Lithuania,10.1,14.27,4.17
Ukraine,10.72,14.46,3.74
Hungary,9.16,12.73,3.57
Germany,8.47,11.42,2.95
Slovenia,8.42,11.37,2.95
Romania,9.14,11.9,2.76
Croatia,9.45,12.18,2.73


The result of our query above shows that Bulgaria is the country which has highest death rate compared to birth rate.